## Parallelize

In [245]:
a = sc.parallelize([1, 2, 3, 4, 5])

In [246]:
a.getNumPartitions()

4

In [247]:
b = a.map(lambda x: 2 * x)

In [248]:
b.collect()

[2, 4, 6, 8, 10]

## Filter

In [249]:
d = a.filter(lambda x: x % 2 == 0)

In [250]:
d.collect()

[2, 4]

## Map FlatMap

In [251]:
e = sc.parallelize(["spark rdd example", "Flat Map for dummies"], 2)

In [252]:
#Operación Map  retornara una array de array
f= e.map(lambda x: x.split(' '))

In [253]:
print(f.collect())

[['spark', 'rdd', 'example'], ['Flat', 'Map', 'for', 'dummies']]


In [254]:
#Operación  Flapmap retornara una array de palabras
f = e.flatMap(lambda x: x.split(' '))

In [255]:
print(f.collect())

['spark', 'rdd', 'example', 'Flat', 'Map', 'for', 'dummies']


## mapPartitions

In [313]:
kv_mpar= sc.parallelize([1, 2, 3, 4, 5, 6],2)
kv_mpar.getNumPartitions()

2

In [314]:
kv_mpar.collect()

[1, 2, 3, 4, 5, 6]

In [257]:
def fun(iterator): yield sum(iterator)

In [258]:
kv_mpar.mapPartitions(fun).collect()

[6, 15]

## mapPartitionsWithIndex

In [259]:
def f(splitIndex, iterator): yield splitIndex

In [260]:
kv_mpar.mapPartitionsWithIndex(f).sum()

1

## mapPartitionsWithSplit

In [261]:
kv_mpar.mapPartitionsWithSplit(f).sum()

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/ipykernel_launcher.py:1: DeprecationWarning: mapPartitionsWithSplit is deprecated; use mapPartitionsWithIndex instead
  """Entry point for launching an IPython kernel.


1

## GroupByKey

In [303]:
kv = sc.parallelize([ ('a', 7), ('b', 4), ('a', 1), ('b', 6), ('c', 3) ])
kv.collect()

[('a', 7), ('b', 4), ('a', 1), ('b', 6), ('c', 3)]

In [304]:
kv_result=kv.groupByKey()

In [305]:
for t in kv_result.collect():
    print(t[0], [v for v in t[1]])

b [4, 6]
c [3]
a [7, 1]


In [306]:
sorted(kv_result.groupByKey().mapValues(list).collect())

[('a', [<pyspark.resultiterable.ResultIterable at 0x10cf30208>]),
 ('b', [<pyspark.resultiterable.ResultIterable at 0x10cf30cc0>]),
 ('c', [<pyspark.resultiterable.ResultIterable at 0x10cf30f98>])]

##  reduceByKey

In [265]:
kv_reduce=kv.reduceByKey(lambda accum, n: accum + n)
print(kv_reduce.collect())

[('b', 10), ('c', 3), ('a', 8)]


## aggregateByKey

In [266]:
studentRDD = sc.parallelize([
('Hugo', 'Escala', 87), 
('Hugo', 'Spark', 93), 
('Hugo', 'Python', 90), 
('Hugo', 'F#', 91),
('Loles', 'Escala', 76), 
('Loles', 'Spark', 79), 
('Loles', 'Python', 78), 
('Loles', 'F#', 88), 
('Tolola', 'Escala', 68), 
('Tolola', 'Spark', 76), 
('Tolola', 'Python', 93), 
('Tolola', 'F#', 90),
('Eustaquio', 'Escala', 61), 
('Eustaquio', 'Spark', 73), 
('Eustaquio', 'Python', 46), 
('Eustaquio', 'F#', 42), 
('Ighor', 'Escala', 59), 
('Ighor', 'Spark', 58), 
('Ighor', 'Python', 72), 
('Ighor', 'F#', 52)  
 ])
#studentRDD.collect()

### la Mejor Nota por alumno

In [267]:
def seqFunc(acc, teams):
    if(acc > teams[1]):
        return acc 
    else: 
        return teams[1]

def combFunc(acc1, acc2):
    if(acc1 > acc2):
        return acc1 
    else:
        return acc2

In [268]:
zero_val = 0
aggr_rdd = studentRDD.map(lambda t: (t[0], (t[1], t[2]))).aggregateByKey(zero_val, seqFunc, combFunc) 

for tpl in aggr_rdd.collect():
    print(tpl) 

('Hugo', 93)
('Tolola', 93)
('Eustaquio', 73)
('Loles', 88)
('Ighor', 72)


### La Mejor Nota de la Materia por Alumno

In [269]:
def seqFuncM(acc, teams):
    if(acc[1] > teams[1]):
        return acc 
    else: 
        return teams

def combFuncM(acc1, acc2):
    if(acc1[1] > acc2[1]):
        return acc1 
    else:
        return acc2

In [270]:
# Zero Value: Zero value in our case will be 0 as we are finding Maximum Marks
zero_val = ('', 0)
aggr_rdd = studentRDD.map(lambda t: (t[0], (t[1], t[2]))).aggregateByKey(zero_val, seqFuncM, combFuncM) 
 
# Check the Outout
for tpl in aggr_rdd.collect():
    print(tpl)
 

('Hugo', ('Spark', 93))
('Tolola', ('Python', 93))
('Eustaquio', ('Spark', 73))
('Loles', ('F#', 88))
('Ighor', ('Python', 72))


### El Promedio de las Notas

In [344]:
def seqFuncP(acc, teams):
    return (acc[0] + teams[1], acc[1] + 1)

def combFuncP(acc1, acc2):
    return (acc1[0] + acc2[0], acc1[1] + acc2[1])

In [345]:
zero_val = (0, 0)
aggr_rdd = studentRDD.map(lambda t: (t[0], (t[1], t[2]))).aggregateByKey(zero_val, seqFuncP, combFuncP).map(lambda t: (t[0], t[1][0]/t[1][1]*1.0))
  
# Check the Outout
for tpl in aggr_rdd.collect():
    print(tpl)

('Hugo', 90.25)
('Tolola', 81.75)
('Eustaquio', 55.5)
('Loles', 80.25)
('Ighor', 60.25)


## sortByKey

In [348]:
aggr_rdd.sortByKey().collect()

[('Eustaquio', 55.5),
 ('Hugo', 90.25),
 ('Ighor', 60.25),
 ('Loles', 80.25),
 ('Tolola', 81.75)]

In [350]:
aggr_rdd.sortByKey(ascending=False).collect()

[('Tolola', 81.75),
 ('Loles', 80.25),
 ('Ighor', 60.25),
 ('Hugo', 90.25),
 ('Eustaquio', 55.5)]

## mapValues

In [287]:
sorted(aggr_rdd.groupByKey().mapValues(len).collect())

[('Eustaquio', 1), ('Hugo', 1), ('Ighor', 1), ('Loles', 1), ('Tolola', 1)]

In [288]:
sorted(aggr_rdd.groupByKey().mapValues(list).collect())

[('Eustaquio', [55.5]),
 ('Hugo', [90.25]),
 ('Ighor', [60.25]),
 ('Loles', [80.25]),
 ('Tolola', [81.75])]

In [297]:
rdd = sc.parallelize([("c", 1), ("b", 1), ("a", 4),("a",2),("b",2)])
sorted(rdd.groupByKey().mapValues(len).collect())




[('a', 2), ('b', 2), ('c', 1)]

In [298]:
sorted(rdd.groupByKey().mapValues(list).collect())

[('a', [4, 2]), ('b', [1, 2]), ('c', [1])]

## Cogroup

In [323]:
kv = sc.parallelize([ ('a', 7), ('b', 4), ('a', 1), ('b', 6), ('c', 3) ])
kv0 = sc.parallelize([('a',5),('b',9),('b',8)])

In [324]:
kv_co=kv.cogroup(kv0)
kv_co.map(lambda x: (x[0],list(x[1][0]),list(x[1][1]))).collect()
 

[('a', [7, 1], [5]), ('b', [4, 6], [9, 8]), ('c', [3], [])]

In [325]:
aggr_rdd=kv_co.map(lambda x: (x[0],list(x[1][0]),list(x[1][1])))
for tpl in aggr_rdd.collect():
    print(tpl)

('a', [7, 1], [5])
('b', [4, 6], [9, 8])
('c', [3], [])


## Join

In [336]:
tkv = spark.createDataFrame(kv,['id','value'])
tkv0 = spark.createDataFrame(kv0,['id','value'])
tkv = tkv.alias('tkv')
tkv0 = tkv0.alias('tkv0')

In [337]:
inner_join = tkv.join(tkv0, tkv.id == tkv0.id)
inner_join.show()

+---+-----+---+-----+
| id|value| id|value|
+---+-----+---+-----+
|  b|    4|  b|    9|
|  b|    4|  b|    8|
|  b|    6|  b|    9|
|  b|    6|  b|    8|
|  a|    7|  a|    5|
|  a|    1|  a|    5|
+---+-----+---+-----+



## Join Left

In [338]:
left_join = tkv.join(tkv0, tkv.id == tkv0.id,how='left')
left_join.show()

+---+-----+----+-----+
| id|value|  id|value|
+---+-----+----+-----+
|  c|    3|null| null|
|  b|    4|   b|    9|
|  b|    4|   b|    8|
|  b|    6|   b|    9|
|  b|    6|   b|    8|
|  a|    7|   a|    5|
|  a|    1|   a|    5|
+---+-----+----+-----+



## Join Left Filter

In [340]:
left_join = tkv.join(tkv0, tkv.id == tkv0.id,how='left')
#left_join.filter(col('tkv0.id').isNull()).show()

## Join  Right

In [341]:
right_join = tkv.join(tkv0, tkv.id == tkv0.id,how='right')
right_join.show()

+---+-----+---+-----+
| id|value| id|value|
+---+-----+---+-----+
|  b|    4|  b|    9|
|  b|    6|  b|    9|
|  b|    4|  b|    8|
|  b|    6|  b|    8|
|  a|    7|  a|    5|
|  a|    1|  a|    5|
+---+-----+---+-----+



## Join  Full

In [342]:
full_join = tkv.join(tkv0, tkv.id == tkv0.id,how='full')
full_join.show()

+---+-----+----+-----+
| id|value|  id|value|
+---+-----+----+-----+
|  c|    3|null| null|
|  b|    4|   b|    9|
|  b|    4|   b|    8|
|  b|    6|   b|    9|
|  b|    6|   b|    8|
|  a|    7|   a|    5|
|  a|    1|   a|    5|
+---+-----+----+-----+

